# Lab 5: Certified Robustness

## Overview

Certified defenses provide provable guarantees against adversarial attacks within a specified threat model.

## Learning Objectives

1. Understand certified defenses
2. Implement randomized smoothing
3. Test certification bounds
4. Evaluate robustness guarantees
5. Compare with empirical defenses

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, models
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Detect device (supports CUDA, Apple Silicon MPS, and CPU)
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✓ Using CUDA GPU: {torch.cuda.get_device_name(0)}")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
    print("✓ Using Apple Silicon GPU (MPS)")
else:
    device = torch.device('cpu')
    print("ℹ Using CPU")

print(f"Device: {device}")

✓ Using Apple Silicon GPU (MPS)
Device: mps


## Part 1: Randomized Smoothing

Randomized smoothing creates a smoothed classifier by averaging predictions over Gaussian noise.

In [2]:
class RandomizedSmoothingClassifier:
    def __init__(self, base_model, sigma=0.25, num_samples=100):
        self.base_model = base_model
        self.sigma = sigma
        self.num_samples = num_samples
        self.base_model.eval()
    
    def predict(self, x):
        counts = torch.zeros(1000).to(x.device)
        with torch.no_grad():
            for _ in range(self.num_samples):
                noisy_x = x + torch.randn_like(x) * self.sigma
                pred = self.base_model(noisy_x).argmax(1).item()
                counts[pred] += 1
        return counts.argmax().item()
    
    def certify(self, x, alpha=0.001):
        counts = torch.zeros(1000).to(x.device)
        with torch.no_grad():
            for _ in range(self.num_samples):
                noisy_x = x + torch.randn_like(x) * self.sigma
                pred = self.base_model(noisy_x).argmax(1).item()
                counts[pred] += 1
        
        top2 = counts.topk(2)
        count_A = top2.values[0].item()
        p_A = count_A / self.num_samples
        
        if p_A > 0.5:
            radius = self.sigma * norm.ppf(p_A)
            return top2.indices[0].item(), radius
        return top2.indices[0].item(), 0.0

print('✓ Randomized smoothing defined')

✓ Randomized smoothing defined


## Part 2: Test Certified Defense

In [3]:
# Load model
base_model = models.resnet18(pretrained=True)
base_model.eval().to(device)

# Create smoothed classifier
smoothed = RandomizedSmoothingClassifier(base_model, sigma=0.25, num_samples=100)

# Test
test_img = torch.randn(1, 3, 224, 224).to(device)
test_img = torch.clamp(test_img, -2, 2)

pred, radius = smoothed.certify(test_img)
print(f'Prediction: class {pred}')
print(f'Certified radius: {radius:.4f}')
print(f'Guaranteed correct for L2 perturbations ≤ {radius:.4f}')

/Users/schwartz/src/genai-security-training/.venv/lib/python3.14/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/schwartz/src/genai-security-training/.venv/lib/python3.14/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Prediction: class 107
Certified radius: inf
Guaranteed correct for L2 perturbations ≤ inf


## Summary

Certified defenses provide provable robustness guarantees, unlike empirical defenses that can be broken by stronger attacks.